# Scratch Pad

This notebook contains functions and code that wasn't used in the final codebase but we wanted to save for reference.

In [ ]:
# Dynamic Bokeh Map

def classMap(path):

    plot = hv.RGB.load_image(path)

    base = tiles.StamenToner

    return base * plot


In [ ]:
# Dynamic Bokeh Map
def dynClassMap(classes):
    points = hv.Points(data, kdims=['x', 'y'], vdims=[classes]).opts(color=classes)
    datashaded  = hd.datashade(points, aggregator=ds.count_cat(classes), 
                               color_key=color_key, dynamic=True,
                              ).opts(height=400, width=800, xaxis=None, yaxis=None, alpha=1)
    final = datashaded
    return final

In [ ]:
# Grid and group
# Create grid of points every 10km
# from https://gis.stackexchange.com/questions/269243/creating-polygon-grid-using-geopandas

xmin,ymin,xmax,ymax = data.total_bounds

cols = list(range(int(np.floor(xmin)), int(np.ceil(xmax)+1000), 10000))
rows = list(range(int(np.floor(ymin)), int(np.ceil(ymax)+1000), 10000))
rows.reverse()

points = []

for x in cols:
    for y in rows:
        points.append(Point(x,y))
        
# Create grid GeoDataFrame
grid = gpd.GeoDataFrame({'geometry': points}, crs={'init':'epsg:6933'})
# grid = grid.to_crs({'init':'epsg:4326'})

grid.crs == data.crs

#Join polygon attributes to the grid
grid_joined = gpd.sjoin(grid, data, how="inner", op='intersects')

# Convert to lat long
grid_joined = grid_joined.to_crs({'init':'epsg:3857'})

grid_joined['x'] = grid_joined.geometry.x
grid_joined['y'] = grid_joined.geometry.y

grid_joined.to_file("./data/grid.shp")

grid_df = pd.DataFrame(grid_joined.drop(columns='geometry')
                      ).rename({'index_right': 'basin_id', 
                                'DBScan_labels': 'label'}, 
                               axis=1)

grid_df.to_csv("./data/dashboard_data.csv")